## Import necessary packages

In [11]:
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={"figure.figsize": (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

from icl_lens_sizing.preprocessing.preprocessing import prepare_training_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load and prepare ICL data

In [2]:
path = "/Users/sortmanns/git/work/icl-lens-sizing/data/icl_data_2023-07-09.csv"
df = pd.read_csv(path, sep=";", decimal=',')
features = ['implantat_size', 'AtA', 'ACW', 'ARtAR_LR', 'StS', 'CBID',
       'WtW_MS-39', 'Sphaere']
# cbid_ratio = (lambda row: row['CBID'] / row['CBID_LR'])
custom_features = None # {'spherical_equivalent': (lambda row: row['Sphaere']-0.5*row['Zylinder'])}
feature_df, target_df, feature_mapping = prepare_training_data(df=df, target="Lens-ICPL-Distance", features=features,custom_features=custom_features)
X_train, X_validation, y_train, y_validation = train_test_split(feature_df, target_df, test_size=0.2)

In [6]:
# Define a range of alpha values
alphas = np.logspace(-4, 2, 10)  # Example range

# Create a dictionary of hyperparameters to search
param_grid = {'estimator__alpha': alphas}

# Create a Lasso regressor
model_lasso = Lasso()
standard_scaler = StandardScaler()
pipeline = Pipeline([('transformer', standard_scaler), ('estimator', model_lasso)])
# Create GridSearchCV or RandomizedSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error')
# For Randomized Search
# random_search = RandomizedSearchCV(lasso, param_distributions=param_grid, cv=5, n_iter=10, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
# For Randomized Search
# random_search.fit(X_train, y_train)
best_alpha = grid_search.best_params_['estimator__alpha']
print("Best alpha:", best_alpha)
best_lasso = Lasso(alpha=best_alpha)
best_lasso.fit(X_train, y_train)
y_pred = best_lasso.predict(X_validation)

mae = mean_absolute_error(y_validation, y_pred)
print("Mean Absolute Error:", mae)

Best alpha: 21.54434690031882
Mean Absolute Error: 121.19245834156754


## Try elastic net on full feature set

In [13]:
features = ['implantat_size', 'alter', 'ACD', 'ACA_nasal', 'ACA_temporal', 'AtA', 'ACW',
            'ARtAR_LR', 'StS', 'StS_LR', 'CBID', 'CBID_LR', 'mPupil', 'WtW_MS-39', 'Sphaere', 'Zylinder']
cbid_ratio = (lambda row: row['CBID'] / row['CBID_LR'])
ac_ratio = (lambda row: row['ACW'] / row['ACD'])
sts_ratio = (lambda row: row['StS'] / row['StS_LR'])
ata_ratio = (lambda row: row['AtA'] / row['ARtAR_LR'])
spherical_equivalent = (lambda row: row['Sphaere']-0.5*row['Zylinder'])
custom_features = {'spherical_equivalent': spherical_equivalent, 'cbid_ratio': cbid_ratio,
                   'ac_ratio':ac_ratio, 'sts_ratio':sts_ratio, 'ata_ratio':ata_ratio}
# Create an Elastic Net regressor
alpha = 0.5  # Regularization strength (mixing parameter)
l1_ratio = 0.5  # Mixing parameter between L1 and L2 regularization
enet = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)

# Fit the model on the training data
enet.fit(X_train, y_train)

# Get the coefficients and intercept
coefficients = enet.coef_
intercept = enet.intercept_

# Obtain p-values using statistical methods (e.g., t-tests)
from scipy import stats

n_samples, n_features = X_train.shape
degrees_of_freedom = n_samples - n_features - 1
t_scores = coefficients / (np.std(y_train) / np.sqrt(np.sum((X_train - np.mean(X_train, axis=0)) ** 2, axis=0)))
p_values = 2 * (1 - stats.t.cdf(np.abs(t_scores), df=degrees_of_freedom))

# Print coefficients and p-values
for i, (coef, p_value) in enumerate(zip(coefficients, p_values)):
    print(f"Feature {i + 1}: Coefficient = {coef:.4f}, P-value = {p_value:.4f}")

Feature 1: Coefficient = -20.9627, P-value = 0.8055
Feature 2: Coefficient = -13.1507, P-value = 0.8492
Feature 3: Coefficient = 17.8742, P-value = 0.7650
Feature 4: Coefficient = -0.5220, P-value = 0.0007
Feature 5: Coefficient = -33.4809, P-value = 0.4659
Feature 6: Coefficient = -52.0218, P-value = 0.2766
Feature 7: Coefficient = -14.0416, P-value = 0.8665
Feature 8: Coefficient = -0.3682, P-value = 0.9462


In [14]:
enet.feature_names_in_

array(['implantat_size', 'AtA', 'ACW', 'ARtAR_LR', 'StS', 'CBID',
       'WtW_MS-39', 'Sphaere'], dtype=object)

In [12]:
# Create a pipeline with standardization and Elastic Net regression
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('enet', ElasticNet())
])

# Define the parameter grid for Elastic Net
param_grid = {
    'enet__alpha': [0.1, 0.5, 1.0],
    'enet__l1_ratio': [0.2, 0.5, 0.8]
}

# Create a custom scorer using mean_absolute_error
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Perform cross-validation with the pipeline
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=mae_scorer)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate on the test set using the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_validation)
test_mae = mean_absolute_error(y_validation, y_pred)
print("Test MAE:", test_mae)

# Alternatively, you can directly use cross_val_score
cross_val_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring=mae_scorer)
print("Cross-Validation MAE Scores:", -cross_val_scores)

Best Hyperparameters: {'enet__alpha': 0.1, 'enet__l1_ratio': 0.5}
Test MAE: 122.70122559761973
Cross-Validation MAE Scores: [101.90645666  81.07458344 111.62651897 186.04806047 160.37383158]
